# Installing Required libraries
Step 1.1- Please refer to the "Technical Requirements" section in the book for the neccessary packages to be installed. Please note that chapter has differnt Pytorch Lightning version and thus diff torch dependancies. Some functions may not work with other versions than what is tested below, so please ensure correct versions.

In [1]:
#Step1 Please refer to the "Technical Requirements" section in the book for the neccessary packages to be installed here


In [2]:
# #refer to book for correct version of package and import here
# !pip install torch==1.10.0 torchvision==0.11.1 torchtext==0.11.0 torchaudio==0.10.0 --quiet
# !pip install pytorch-lightning==1.5.2 --quiet




# Step 1.2 Import the neccessary packages.
 Please refer to the section, Importing the packages in the book for further details. 

In [1]:
import torch
from torch import nn, optim
from torch.autograd import Variable
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
from torchmetrics.functional import accuracy


In [2]:
print("torch version:",torch.__version__)
print("pytorch ligthening version:",pl.__version__)

torch version: 2.2.1
pytorch ligthening version: 2.2.1


In [3]:
xor_input = [Variable(torch.Tensor([0, 0])),
           Variable(torch.Tensor([0, 1])),
           Variable(torch.Tensor([1, 0])),
           Variable(torch.Tensor([1, 1]))]

In [4]:
xor_target = [Variable(torch.Tensor([0])),
           Variable(torch.Tensor([1])),
           Variable(torch.Tensor([1])),
           Variable(torch.Tensor([0]))]

In [6]:
xor_data = list(zip(xor_input, xor_target))
train_loader = DataLoader(xor_data, batch_size=1000)

[(tensor([0., 0.]), tensor([0.])),
 (tensor([0., 1.]), tensor([1.])),
 (tensor([1., 0.]), tensor([1.])),
 (tensor([1., 1.]), tensor([0.]))]

In [7]:
class XORModel(pl.LightningModule):
  def __init__(self):

    super(XORModel,self).__init__()
    
    self.input_layer = nn.Linear(2, 4)
    self.output_layer = nn.Linear(4,1)

    self.sigmoid = nn.Sigmoid()

    self.loss = nn.MSELoss()

  def forward(self, input):
    #print("INPUT:", input.shape)
    x = self.input_layer(input)
    #print("FIRST:", x.shape)
    x = self.sigmoid(x)
    #print("SECOND:", x.shape)
    output = self.output_layer(x)
    #print("THIRD:", output.shape)
    return output

  def configure_optimizers(self):
    params = self.parameters()
    optimizer = optim.Adam(params=params, lr = 0.01)
    return optimizer

  def training_step(self, batch, batch_idx):
    xor_input, xor_target = batch
    #print("XOR INPUT:", xor_input.shape)
    #print("XOR TARGET:", xor_target.shape)
    outputs = self(xor_input) 
    #print("XOR OUTPUT:", outputs.shape)
    loss = self.loss(outputs, xor_target)
    return loss 

In [11]:
from pytorch_lightning.utilities.types import TRAIN_DATALOADERS
checkpoint_callback = ModelCheckpoint()
model = XORModel()

trainer = pl.Trainer(max_epochs=100, callbacks=[checkpoint_callback])

trainer.fit(model, train_dataloaders=train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name         | Type    | Params
-----------------------------------------
0 | input_layer  | Linear  | 12    
1 | output_layer | Linear  | 5     
2 | sigmoid      | Sigmoid | 0     
3 | loss         | MSELoss | 0     
-----------------------------------------
17        Trainable params
0         Non-trainable params
17        Total params
0.000     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [12]:
!ls lightning_logs/

version_0 version_1


In [13]:
!ls lightning_logs/*/

lightning_logs/version_0/:
checkpoints
events.out.tfevents.1710700440.MacBook-Pro.local.14143.0
hparams.yaml

lightning_logs/version_1/:
checkpoints
events.out.tfevents.1710700743.MacBook-Pro.local.14143.1
hparams.yaml


In [14]:
print(checkpoint_callback.best_model_path)

/Users/pushkarsinha/Documents/git/external/Deep-Learning-with-PyTorch-Lightning/Chapter02/lightning_logs/version_1/checkpoints/epoch=99-step=100.ckpt


In [16]:
print(checkpoint_callback.best_model_path)
train_model = XORModel.load_from_checkpoint(checkpoint_callback.best_model_path)
test = torch.utils.data.DataLoader(xor_input, batch_size=1)
for val in xor_input:
  _ = train_model(val)
  print([int(val[0]),int(val[1])], int(_.round()))

/Users/pushkarsinha/Documents/git/external/Deep-Learning-with-PyTorch-Lightning/Chapter02/lightning_logs/version_1/checkpoints/epoch=99-step=100.ckpt


[0, 0] 0
[0, 1] 0
[1, 0] 1
[1, 1] 1


In [17]:
print(checkpoint_callback.best_model_path)
train_model = XORModel.load_from_checkpoint(checkpoint_callback.best_model_path)
total_accuracy = []
for xor_input, xor_target in train_loader:
  for i in range(100):
    output_tensor = train_model(xor_input)
    test_accuracy = accuracy(output_tensor, xor_target.int())
    total_accuracy.append(test_accuracy)
total_accuracy = torch.mean(torch.stack(total_accuracy))
print("TOTAL ACCURACY FOR 100 ITERATIONS: ", total_accuracy.item())

/Users/pushkarsinha/Documents/git/external/Deep-Learning-with-PyTorch-Lightning/Chapter02/lightning_logs/version_1/checkpoints/epoch=99-step=100.ckpt


TypeError: accuracy() missing 1 required positional argument: 'task'